# DATAFRAME FILTER FUNCTION

How it works:  
The function takes the rating.csv file and filter it based on the number of occurences of userId or movieId.  
We created this function to be able to work on the exact same filtered dataframes from remote locations, just sharing the set of chosen parameters.  

Parameters:  

rating_df (DataFrame): The input DataFrame to be filtered. - (DO NOT CHANGE IT)  
movie_df (DataFrame): The DataFrame containing movie details. - (DO NOT CHANGE IT)  
col_name (str): Choose to count occurences of userId or movieId.  
lower_bound (int): The lower bound for the number of ratings to filter.  
upper_bound (int): The upper bound for the number of ratings to filter.  
output_filename (str): The base name of the output CSV files (without '.csv').

In [2]:
# Import Pandas Library
import pandas as pd

In [3]:
# Set path to csv
path_rating = "input_data\\rating.csv"
path_movie = "input_data\\movie.csv"

In [4]:
# Loading rating dataframe
rating_df = pd.read_csv(path_rating)


In [5]:
# Loading movie dataframe
movie_df = pd.read_csv(path_movie)

# Use str.extract to separate the movie title and year from the 'title' column
movie_df[['title', 'year']] = movie_df['title'].str.extract(r'^(.*)\s\((\d{4})\)$')

# Perform one-hot encoding on the 'genres' column, creating binary columns for each genre
df_genres_encoded = movie_df['genres'].str.get_dummies(sep='|')

# Concatenate the one-hot encoded genres with the original DataFrame
movie_df = pd.concat([movie_df, df_genres_encoded], axis=1)

# Drop the 'genres', 'year', and '(no genres listed)' columns from the DataFrame
movie_df = movie_df.drop(columns=['(no genres listed)'], axis=1)


#### Filter Function

In [6]:
def filter_and_save_dataframes(rating_df, movie_df, col_name, lower_bound, upper_bound, output_filename):
    """
    Filters the DataFrame based on the number of ratings and writes the result to two CSV files.
    
    Parameters:
    rating_df (DataFrame): The input DataFrame to be filtered.
    movie_df (DataFrame): The DataFrame containing movie details.
    col_name (str): The name of the column to count ratings per userId or movieId
    lower_bound (int): The lower bound for the number of ratings to filter.
    upper_bound (int): The upper bound for the number of ratings to filter.
    output_filename (str): The base name of the output CSV files (without '.csv').
    
    Returns:
    DataFrame: The filtered DataFrame.
    """
    # Calculate the number of ratings each movie or user has received
    rating_df[f'ratings_per_{col_name}'] = rating_df.groupby(col_name)['userId'].transform('count')
    
    # Filter the DataFrame
    filtered_df = rating_df[(rating_df[f'ratings_per_{col_name}'] > lower_bound) & 
                     (rating_df[f'ratings_per_{col_name}'] < upper_bound)]
    
    # Drop all columns except 'userId', 'movieId', 'rating'
    filtered_df = filtered_df[['userId', 'movieId', 'rating']]
    
    # Print new dataset name
    print("Name : ",output_filename, "\n")
    
    # Print the number and percentage of ratings in the filtered DataFrame
    print("nb of ratings :",len(filtered_df))
    print("percentage of ratings :", round(len(filtered_df)/len(rating_df)*100),"%\n")

    # Print the number and percentage of unique users in the filtered DataFrame
    print("nb of users :",filtered_df['userId'].nunique())
    print("percentage of users :", round(filtered_df['userId'].nunique()/rating_df['userId'].nunique()*100),"%\n")

    # Print the number and percentage of unique movies in the filtered DataFrame
    print("nb of movies :",filtered_df['movieId'].nunique())
    print("percentage of movies :", round(filtered_df['movieId'].nunique()/rating_df['movieId'].nunique()*100),"%\n")
    
    # Write the filtered ratings DataFrame into a CSV
    filtered_df.to_csv(f'rating_{output_filename}.csv', index=False)
    
    # Dynamically set the name of the DataFrame
    globals()[output_filename] = filtered_df
    
    # Filter movie_df to keep only the movieIds present in filtered_df
    filtered_movie_df = movie_df[movie_df['movieId'].isin(filtered_df['movieId'])]
    
    print("Do the rating filtered df and the movie filtered df contain the sames movies?")
    # Check if the number of unique movieIds matches
    if len(filtered_movie_df) == filtered_df['movieId'].nunique():
        print("Success: The lengths match.")
    else:
        print("Error: Lengths do not match.")
    
    # Write the filtered movie DataFrame into a CSV
    filtered_movie_df.to_csv(f'movie_{output_filename}.csv', index=False)
    
    
    # Return the filtered DataFrame for further use if needed
    return filtered_df, filtered_movie_df


# Call the combined function
filter_and_save_dataframes(rating_df, movie_df, 'userId', 123, 644, 'IQR_users')

"""
Tips to use the function
userId rating count range: 
mean       511
std        663
min         20
25%        123
50%        296
75%        644
max       9254

movieId rating count range:
mean       747
std       3085
min          1
25%          3
50%         18
75%        205
max      67310

Exemple of filtering we have used with our models:

Blockbusters: very popular movies
(rating_df, movie_df, 'movieId', 1300, 67311, 'blockbusters_movies')

Confidentials: movies not so popular
(rating_df, movie_df, 'movieId', 0, 1300, 'confidential_movies')

Engaged: Very active users
(rating_df, movie_df, 'userId', 644, 9255, 'engaged_users')

New Comers : Poorly active users
(rating_df, movie_df, 'userId', 0, 123, 'newcomers_users')

IQR: average active users, count for 50% of all ratings
(rating_df, movie_df, 'userId', 123, 644, 'IQR_users')
"""


Name :  IQR_users 

nb of ratings : 9962336
percentage of ratings : 50 %

nb of users : 38105
percentage of users : 28 %

nb of movies : 19658
percentage of movies : 74 %

Do the rating filtered df and the movie filtered df contain the sames movies?
Success: The lengths match.


"\nTips to use the function\nuserId rating count range: \nmean       511\nstd        663\nmin         20\n25%        123\n50%        296\n75%        644\nmax       9254\n\nmovieId rating count range:\nmean       747\nstd       3085\nmin          1\n25%          3\n50%         18\n75%        205\nmax      67310\n\nExemple of filtering we have used with our models:\n\nBlockbusters: very popular movies\n(rating_df, movie_df, 'movieId', 1300, 67311, 'blockbusters_movies')\n\nConfidentials: movies not so popular\n(rating_df, movie_df, 'movieId', 0, 1300, 'confidential_movies')\n\nEngaged: Very active users\n(rating_df, movie_df, 'userId', 644, 9255, 'engaged_users')\n\nNew Comers : Poorly active users\n(rating_df, movie_df, 'userId', 0, 123, 'newcomers_users')\n\nIQR: average active users, count for 50% of all ratings\n(rating_df, movie_df, 'userId', 123, 644, 'IQR_users')\n"